# Anexo 3 GEMA

Solo válido para vias no pavimentadas

## Datos de proyecto en análisis

In [197]:
proyecto = "ccanaseta"
mes = 4
anio = 2025

## Importar librerias

In [198]:
from dotenv import load_dotenv, find_dotenv
from dotenv import load_dotenv, find_dotenv
from google.cloud import firestore
from google.oauth2 import service_account
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from pathlib import Path
from typing import Dict, List, Any
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import excel2img
import itertools
import json
import math
import os
import pandas as pd
import pickle
import pprint
import random
import re
import win32com.client as win32
import xlsxwriter

from typing import TypedDict, List, Dict, Optional

## Fuciones utiles

### Generador de errores

In [199]:
def generar_datos_errores_unidades(
    longitud: float,
    error_permitido: float,
    recurrencia: float,
    intensidad: float,
    variacion: float,
    paso: float,
    seed: Optional[int] = None
) -> List[Dict]:
    """
    Genera datos falsos de errores distribuidos en tramos de carretera.

    Parámetros:
    - longitud (float): Longitud total de la carretera en metros. Debe ser > 0.
    - error_permitido (float): Número máximo de fallas por kilómetro (contador ≥ 0).
    - recurrencia (float): Proporción de tramos que tendrán error (0 ≤ recurrencia ≤ 1).
    - intensidad (float): Factor que multiplica el máximo de errores permitidos (≥ 0).
    - variacion (float): Amplitud de variación relativa sobre el valor base (0 ≤ variacion ≤ 1).
    - paso (float): Incremento mínimo para el valor de error (> 0).
    - seed (int, opcional): Semilla para reproducibilidad. Por defecto None.

    Validaciones:
    - longitud > 0
    - error_permitido ≥ 0
    - 0 ≤ recurrencia ≤ 1
    - intensidad ≥ 0
    - 0 ≤ variacion ≤ 1
    - paso > 0

    Salida:
    Lista de diccionarios, uno por tramo. Cada dict contiene:
    - "tramo": número de tramo (1, 2, …)
    - "inicio_m": punto de inicio del tramo (en metros)
    - "fin_m": punto final del tramo (en metros)
    - "longitud_m": longitud efectiva del tramo (m)
    - "error": valor simulado de errores (múltiplo de `paso`)
    """
    # Validaciones de entrada
    if longitud <= 0:
        raise ValueError("La longitud debe ser > 0.")
    if error_permitido < 0:
        raise ValueError("error_permitido debe ser ≥ 0.")
    if not (0 <= recurrencia <= 1):
        raise ValueError("recurrencia debe estar entre 0 y 1.")
    if intensidad < 0:
        raise ValueError("intensidad debe ser ≥ 0.")
    if not (0 <= variacion <= 1):
        raise ValueError("variacion debe estar entre 0 y 1.")
    if paso <= 0:
        raise ValueError("paso debe ser > 0.")

    # Fijar semilla si se pide
    if seed is not None:
        random.seed(seed)

    # División en tramos de 1 km
    num_tramos = math.ceil(longitud / 1000)
    tramos = []
    for i in range(num_tramos):
        inicio = i * 1000.0
        fin = min((i + 1) * 1000.0, longitud)
        tramos.append({
            "tramo": i + 1,
            "inicio_m": inicio,
            "fin_m": fin,
            "longitud_m": fin - inicio,
            "error": 0.0  # se llenará luego
        })

    # Número de tramos con error
    errores_tramos = round(num_tramos * recurrencia)

    # Selección "espaciada" de índices
    indices_error = set()
    if errores_tramos > 0:
        paso_espacio = num_tramos / errores_tramos
        for k in range(errores_tramos):
            idx = int(math.floor((k + 0.5) * paso_espacio))
            # asegurar rango válido
            idx = max(0, min(num_tramos - 1, idx))
            indices_error.add(idx)

    # Generación de valores de error
    base_error = intensidad * error_permitido
    for idx in indices_error:
        # variación aleatoria en ±variacion * base_error
        delta = random.uniform(-variacion, variacion) * base_error
        raw = base_error + delta
        # redondear al múltiplo de `paso`
        ajustado = round(raw / paso) * paso
        # evitar negativos
        tramos[idx]["error"] = max(0.0, ajustado)

    return tramos

## Carga de datos

### Firebase

In [200]:
# 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError(
        "No se encontró ningún archivo .env en este directorio ni en sus padres."
    )
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [201]:
db = firestore.Client(credentials=credentials, project=credentials.project_id)

In [202]:
doc_proyecto_firebase = db.collection("rutinarios").document(proyecto).get().to_dict()

pprint.pprint(doc_proyecto_firebase)

{'contrato': {'contratista': {'razon_social': 'Grupo ARICOL E.I.R.L',
                              'ruc': 20606988398},
              'denominacion_tramo_convenio': 'EMP. PE-34 E - CCANASETA (KM '
                                             '8+170)',
              'fecha_inicio': {'anio': 2025, 'dia': 16, 'mes': 4},
              'id_contrato': '003-2025',
              'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                                     'dni': 0,
                                     'nombre': 'Genaro',
                                     'titulo': 'Ingeniero'},
              'monto_contrato': 40784.8,
              'numero_cuadrillas': 1,
              'numero_trabajadores': 3,
              'tiempo_ejecucion_dias': 240,
              'tipo_servicio': 'mantenimiento rutinario'},
 'datos_generales': {'distritos': ['Callalli'],
                     'provincia': 'Caylloma',
                     'region': 'Arequipa'},
 'expediente': {'codigo_ruta': 'AR-691',
       

In [203]:
expediente=doc_proyecto_firebase['expediente']
longitud=expediente['longitud']

print(longitud)

8170.0


### Raw

In [204]:
actividades={
    '101':{
        "error_permitido":3,
        "recurrencia":0.8,
        "intensidad":0.5,
        "variacion":0.1,
        "paso":1,
    },
    '102':{
        "error_permitido":10,
        "recurrencia":0.25,
        "intensidad":0.25,
        "variacion":0.25,
        "paso":1,
    },
    '104':{
        "error_permitido":1,
        "recurrencia":0.9,
        "intensidad":0.4,
        "variacion":0.24,
        "paso":0.1,
    },
    '201':{
        "error_permitido":25,
        "recurrencia":0.8,
        "intensidad":0.75,
        "variacion":0.1,
        "paso":1,
    },
    '203':{
        "error_permitido":30,
        "recurrencia":0.8,
        "intensidad":0.5,
        "variacion":0.1,
        "paso":1,
    },
    '301':{
        "error_permitido":45,
        "recurrencia":0.6,
        "intensidad":0.6,
        "variacion":0.1,
        "paso":1,
    },
    '401':{
        "error_permitido":1,
        "recurrencia":0.15,
        "intensidad":1,
        "variacion":0,
        "paso":0.1,
    },
    '701':{
        "error_permitido":5,
        "recurrencia":0.25,
        "intensidad":0.25,
        "variacion":0.4,
        "paso":0.25,
    },
}

## Calculos

In [225]:
actividad_en_curso = '203'  # Por ejemplo, la actividad 101

In [226]:
error= actividades[actividad_en_curso]["error_permitido"]
recurrencia = actividades[actividad_en_curso]["recurrencia"]
intensidad = actividades[actividad_en_curso]["intensidad"]
variacion = actividades[actividad_en_curso]["variacion"]
paso = actividades[actividad_en_curso]["paso"]

In [227]:
datos_unidades = generar_datos_errores_unidades(
    longitud=longitud,
    error_permitido=error,
    recurrencia=recurrencia,
    intensidad=intensidad,
    variacion=variacion,
    paso=paso,
    # seed=23
)

In [228]:
texto_para_excel = '\t'.join(str(round(item['error'], 2)) for item in datos_unidades)

# Mostrar sin print para facilitar copiar
from IPython.display import display, HTML
display(HTML(f"<textarea rows=3 cols=100>{texto_para_excel}</textarea>"))